In [8]:
import datetime#import datetime module convert data to datetime64(ns)
import numpy as np
import pandas as pd#import pandas module(Better than csv module!)
import os
import glob
from sklearn.model_selection import train_test_split
global train_x , train_y,x_test,y_test,TimeData,Type
train_x=[]
train_y=[]
x_test=[]
y_test=[]

def Resha(x):
    nptrain=np.array(x)
    print(nptrain.shape)
    nptrain=np.reshape(nptrain,(nptrain.shape[0]*nptrain.shape[1], nptrain.shape[2]))
    #print(nptrain)
    return nptrain

def Savenp(x,y):
    
    train_x,x_test,train_y,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
    
    nptrain=Resha(train_x)
    np.save(os.path.join('./StockData/TrainingData/','NormtrainingX_'+path[12:21]),nptrain)
    print(path[12:21],"trainx",nptrain.shape)
    
    nptrain=Resha(x_test)
    np.save(os.path.join('./StockData/TrainingData/','NormtestingX_'+path[12:21]),nptrain)
    print(path[12:21],"testx",nptrain.shape)
    
    nptrain=np.array(train_y)
    np.save(os.path.join('./StockData/TrainingData/','trainingY_'+path[12:21]),nptrain)
    print(path[12:21],"trainy",nptrain.shape)
    
    nptrain=np.array(y_test)
    np.save(os.path.join('./StockData/TrainingData/','testingY_'+path[12:21]),nptrain)
    print(path[12:21],"testy",nptrain.shape)

#def add_features()
    
def GenerateTrain(Type,feature):
    for tmp in TimeData:#print(tmp[1])
        i=tmp[1].dropna()
        i=i.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
        #print(i)
        if len(i)==5: #Decide the way seperate the stock data
            train_x.append(i.loc[ : ,feature].values.tolist())
            Open=i.loc[ : ,'open'].values.tolist()[0]
            Close=i.loc[ : ,Type].values.tolist()[len(i.loc[ : ,Type].values.tolist())-1]
            train_y.append(float(Close-Open))
        else:
            continue
    Savenp(train_x,train_y)

feature=input("Enter the needed features seperated with space: ").split(" ")
closeType=input("Enter the close type: ")
resam=input("Enter the desired  data span , d  for day  w  for week  m  for month: ")
#span=int(input("Enter the span of trainx trainy: "))

for path in glob.glob(r'./StockData/stock0050.csv'):
    CsvData=pd.DataFrame(pd.read_csv(path))
    CsvData['date']=pd.to_datetime(CsvData['date'])
    CsvData=CsvData.drop([0],axis=0)#drop 第一天 因為stockdata 有16年跳到17年的問題
   #print(CsvData)
    #CsvData = add_features(CsvData)
    TimeData=CsvData.set_index('date').resample(resam)
    GenerateTrain(closeType,feature)
    train_x.clear()
    train_y.clear()
    '''CsvData["week"]=0 #Csvdata type dataframe
    for i,j in zip(CsvData["date"],range(CsvData["date"].size)):
        CsvData.loc[j:j+1,"week"]=i.isocalendar()[2]'''

Enter the needed features seperated with space: close open volume low high
Enter the close type: close
Enter the desired  data span , d  for day  w  for week  m  for month: w
(503, 5, 5)
stock0050 trainx (2515, 5)
(168, 5, 5)
stock0050 testx (840, 5)
stock0050 trainy (503,)
stock0050 testy (168,)
